***GENERATED CODE FOR creditcardclassf130aug PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://backendmaster:7077').getOrCreate()
#%run creditcardclassf130augHooks.ipynb
try:
	#sourcePreExecutionHook()

	creditcardtraindata = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1692339008957_creditcardTrainData.csv', 'filename': 'creditcardTrainData.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'FilePath': '/Finance/Credit Card/creditcardTrainData.csv', 'viewFileName': 'creditcardTrainData.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run creditcardclassf130augHooks.ipynb
try:
	#transformationPreExecutionHook()

	creditcardclassfaugautofe = TransformationMain.run(creditcardtraindata,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "Time", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "89331.5", "stddev": "199.27", "min": "88999.0", "max": "89685.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Time"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V1", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.15", "stddev": "1.24", "min": "-5.852465108", "max": "1.586092522", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V2", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.22", "stddev": "1.11", "min": "-5.931717441", "max": "5.26737615", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V3", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.64", "stddev": "1.16", "min": "-7.572355818", "max": "3.194595344", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V4", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.31", "stddev": "1.22", "min": "-4.657545034", "max": "3.997905588", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V4"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V5", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.03", "stddev": "1.05", "min": "-3.125908514", "max": "7.672543966", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V5"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V6", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.02", "stddev": "1.38", "min": "-2.89524724", "max": "6.089350985", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V6"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V7", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.08", "stddev": "0.84", "min": "-4.925568043", "max": "4.185906257", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V7"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V8", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.07", "stddev": "0.85", "min": "-7.494657608", "max": "1.726412535", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V8"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V9", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.2", "stddev": "0.97", "min": "-3.74059751", "max": "4.802464879", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V9"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V10", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.17", "stddev": "1.11", "min": "-3.651810546", "max": "11.15397637", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V10"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V11", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.26", "stddev": "0.89", "min": "-3.04610694", "max": "1.664526627", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V11"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V12", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.25", "stddev": "0.82", "min": "-8.707879292", "max": "4.403898555", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V12"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V13", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.03", "stddev": "0.93", "min": "-2.899032811", "max": "2.361502055", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V13"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V14", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.0", "stddev": "0.86", "min": "-9.577194393", "max": "7.421943853", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V14"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V15", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.24", "stddev": "0.93", "min": "-3.618059825", "max": "2.501394702", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V15"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V16", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.05", "stddev": "0.92", "min": "-3.533506913", "max": "3.594874683", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V16"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V17", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.09", "stddev": "0.81", "min": "-12.50393097", "max": "2.758454159", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V17"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V18", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.02", "stddev": "0.88", "min": "-2.66186027", "max": "2.584307206", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V18"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V19", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.02", "stddev": "0.84", "min": "-2.452829938", "max": "3.078990407", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V19"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V20", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.06", "stddev": "0.98", "min": "-10.06949862", "max": "10.23315086", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V20"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V21", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.09", "stddev": "0.52", "min": "-2.963596118", "max": "3.944361437", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V21"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V22", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.13", "stddev": "0.6", "min": "-2.331390339", "max": "1.461535075", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V22"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V23", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.04", "stddev": "0.32", "min": "-1.786328942", "max": "3.150413499", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V23"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V24", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.03", "stddev": "0.63", "min": "-1.867208192", "max": "1.108439997", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V24"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V25", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.11", "stddev": "0.43", "min": "-1.389079168", "max": "1.125918008", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V25"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V26", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.06", "stddev": "0.48", "min": "-1.228422571", "max": "1.194424515", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V26"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V27", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.01", "stddev": "0.35", "min": "-2.639966768", "max": "1.795968246", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V27"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V28", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.01", "stddev": "0.27", "min": "-2.735622762", "max": "1.573083584", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V28"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "87.23", "stddev": "249.47", "min": "0.0", "max": "4102.74", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Class", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "Class"}]}))

	#transformationPostExecutionHook(creditcardclassfaugautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run creditcardclassf130augHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(creditcardclassfaugautofe, ["Time", "V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9", "V10", "V11", "V12", "V13", "V14", "V15", "V16", "V17", "V18", "V19", "V20", "V21", "V22", "V23", "V24", "V25", "V26", "V27", "V28", "Amount"], "Class")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

